### GOAL

Build one tiny backend resource using correct thinking order.

In [1]:
"""
    1. Resources -> User
"""

'\n    1. Resources -> User\n'

In [2]:
"""
    2. Database Model
"""

import uuid
from sqlmodel import Field, SQLModel

class User(SQLModel, table=True):
    __tablename__="users"
    uid: uuid.UUID | None = Field(default=uuid.uuid4, primary_key=True)
    email : str = Field(..., unique=True, nullable=False, index=True)
    full_name: str = Field(...)
    hash_psswd: str = Field(..., nullable=False)

In [ ]:
"""
    3. Schema
"""
from typing import Optional
import uuid
from pydantic import BaseModel

class UserCreate(BaseModel):
    email: str
    full_name: str
    hash_psswd: str
    
class UserRead(BaseModel):
    uid: uuid.UUID
    email: str
    full_name: str
    
class UserUpdate(BaseModel):
    email: str
    full_name: str
    password: str

class UserPatchUpdate(BaseModel):
    email: Optional[str] = None
    full_name: Optional[str] = None
    password: Optional[str] = None


In [ ]:
"""
    4. Service
"""

from fastapi import HTTPException, status
from sqlmodel import select
from sqlalchemy.ext.asyncio import AsyncSession


class UserService:
    async def create(self, payload: UserCreate, session: AsyncSession):
        statement = select(User).where(User.email == payload.email)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if user:
            return {"error":"User already exists"}
        
        new_user = User(email = payload.email,
                        full_name = payload.full_name,
                        password = payload.hash_psswd)
        
        
        session.add(new_user)
        await session.commit()
        await session.refresh(new_user)
        
        return new_user
    
    
    async def get_user_by_id(self, user_id: uuid.UUID, session: AsyncSession):
        statement = select(User).where(User.uid == user_id)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if not user:
            raise ValueError("User not found")
        
        return user
    
    
    async def get_all_users(self, session: AsyncSession, limit:int = 10, offset: int = 0):
        statement = select(User).limit(limit).offset(offset)
        resutl = await session.execute(statement)
        all_users = resutl.scalars().all()
        
        if not all_users:
            raise ValueError("Users not found")
        
        return all_users
    
    
    async def update_all_details(self, user_id: uuid.UUID, payload: UserUpdate, session: AsyncSession):
        statement = select(User).where(User.uid == user_id)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if not user:
            raise ValueError("Users not found")
        
        user.email=payload.email
        user.full_name=payload.full_name
        user.hash_psswd=payload.password
        
        session.add(user)
        await session.commit()
        await session.refresh(user)
                               
        
        return user
    
    async def update_patch(self, user_uid: uuid.UUID, payload: UserPatchUpdate, session: AsyncSession):
        statement = select(User).where(User.uid == user_uid)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if not user :
            raise ValueError("Users not found")
        
        if not any([
            payload.email,
            payload.full_name,
            payload.password
        ]):
            raise ValueError("No fields provided for update")

        
        if user.email is not None:
            user.email = payload.email
            
        if user.full_name is not None:
            user.full_name = payload.full_name
            
        if user.hash_psswd is not None:
            user.hash_psswd = hash_password(payload.password)
            
        session.add(user)
        await session.commit()
        await session.refresh(user)
        
        return user
    
    
    async def delete_usr(self, user_id: uuid.UUID, session: AsyncSession):
        statement = select(User).where(User.uid == user_id)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if not user:
            raise ValueError("Users not found")
        
        session.delete(user)
        await session.commit()
        return ("message":"User deleted")
    

In [5]:
def get_session(ab):
    pass

In [ ]:
"""
    4. Routes (REST Endpoints)
"""


from fastapi import FastAPI, Query, status, Depends, HTTPException
from sqlalchemy.ext.asyncio import AsyncSession

app = FastAPI()
user_service = UserService()

'''Creating User'''
@app.post("/user/create", response_model=UserRead, status_code=status.HTTP_201_CREATED)
async def create_user(payload: UserCreate, session: AsyncSession = Depends(get_session)):
    try:
        user = await user_service.create(payload, session)
        return user
    
    except ValueError as e:
        raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail=str(e))



'''Fetching the user'''    
@app.get("/user/{user_id}", response_model=UserRead, status_code=status.HTTP_200_OK)
async def get_user(user_id: uuid.UUID, session: AsyncSession = Depends(get_session)):
    user = await user_service.get_user_by_id(user_id, session)
    
    if user is None:
        return []
    
    return user



'''Fetch all users'''
@app.get("/users/", response_model=list[UserRead], status_code=status.HTTP_200_OK)
async def get_all_user(session: AsyncSession = Depends(get_session),
                       limit: int = Query(10, ge=1, le=100),
                       offset: int = Query(0, ge=0)):
    try:
        all_user = await user_service.get_all_users(session=session, limit=limit, offset=offset)
        return all_user
    
    except Exception as e:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Users not found")
    
    
'''Update all fields'''
@app.put("/users/{user_id}", response_model=UserRead, status_code=status.HTTP_200_OK)
async def update_all(user_id: uuid.UUID, payload: UserUpdate, session: AsyncSession = Depends(get_session)):
        try:
            update_detail = await user_service.update_all_details(user_id, payload, session)
            return update_detail
        
        except Exception as e:
            raise HTTPException(status_code=404, detail=str(e))
        
        
'''Update the specific field'''
@app.patch("/users/{user_id}", response_model=UserRead, status_code=status.HTTP_200_OK)
async def update_patch(user_id: uuid.UUID, payload: UserPatchUpdate, session: AsyncSession = Depends(get_session)):
    try:
        update_field = await user_service.update_patch(user_id, payload, session)
        return update_field
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))


'''Delete user'''
@app.delete("/user/{user_id}", status_code=status.HTTP_200_OK)
async def delete_users(user_id: uuid.UUID, session: AsyncSession = Depends(get_session)):
    try:
        deleted = await user_service.delete_usr(user_id, session)
        return {"message":"User deleted"}
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))